In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spacy
import empath
from util import const

In [3]:
fm = const.FileManager()

In [6]:
nlp = spacy.load('en')

In [7]:
doc = nlp('this is a sentence. Actually, two sentences')

In [21]:
doc.print_tree()[0]

/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py:193: DeprecationWarning: [W003] Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)


{'word': 'is',
 'lemma': 'be',
 'NE': '',
 'POS_fine': 'VBZ',
 'POS_coarse': 'VERB',
 'arc': 'ROOT',
 'modifiers': [{'word': 'this',
   'lemma': 'this',
   'NE': '',
   'POS_fine': 'DT',
   'POS_coarse': 'DET',
   'arc': 'nsubj',
   'modifiers': []},
  {'word': 'sentence',
   'lemma': 'sentence',
   'NE': '',
   'POS_fine': 'NN',
   'POS_coarse': 'NOUN',
   'arc': 'attr',
   'modifiers': [{'word': 'a',
     'lemma': 'a',
     'NE': '',
     'POS_fine': 'DT',
     'POS_coarse': 'DET',
     'arc': 'det',
     'modifiers': []}]},
  {'word': '.',
   'lemma': '.',
   'NE': '',
   'POS_fine': '.',
   'POS_coarse': 'PUNCT',
   'arc': 'punct',
   'modifiers': []}]}

# Empath
Lexicon substitute for the commercial LIWC.

In [23]:
lexicon = empath.Empath()

In [26]:
lexicon.analyze('he hit the other person', normalize=True)

{'help': 0.0,
 'office': 0.0,
 'dance': 0.0,
 'money': 0.0,
 'wedding': 0.0,
 'domestic_work': 0.0,
 'sleep': 0.0,
 'medical_emergency': 0.0,
 'cold': 0.0,
 'hate': 0.0,
 'cheerfulness': 0.0,
 'aggression': 0.0,
 'occupation': 0.0,
 'envy': 0.0,
 'anticipation': 0.0,
 'family': 0.0,
 'vacation': 0.0,
 'crime': 0.0,
 'attractive': 0.0,
 'masculine': 0.0,
 'prison': 0.0,
 'health': 0.0,
 'pride': 0.0,
 'dispute': 0.0,
 'nervousness': 0.0,
 'government': 0.0,
 'weakness': 0.0,
 'horror': 0.0,
 'swearing_terms': 0.0,
 'leisure': 0.0,
 'suffering': 0.0,
 'royalty': 0.0,
 'wealthy': 0.0,
 'tourism': 0.0,
 'furniture': 0.0,
 'school': 0.0,
 'magic': 0.0,
 'beach': 0.0,
 'journalism': 0.0,
 'morning': 0.0,
 'banking': 0.0,
 'social_media': 0.0,
 'exercise': 0.0,
 'night': 0.0,
 'kill': 0.0,
 'blue_collar_job': 0.0,
 'art': 0.0,
 'ridicule': 0.0,
 'play': 0.0,
 'computer': 0.0,
 'college': 0.0,
 'optimism': 0.0,
 'stealing': 0.0,
 'real_estate': 0.0,
 'home': 0.0,
 'divine': 0.0,
 'sexual': 0.0

# Positive/Negative words

Positive words in ASCII, negative ISO-8859. `;` indicates comments, one empty line between comments and start of words.

In [4]:
def words_from_file(fname, encoding='UTF-8'):
    with open(fname, encoding=encoding) as f:
        s = set()
        include = lambda w: not (len(w.strip()) == 0 or w.startswith(';'))
        return set(w.strip().lower() for w in f if include(w))
    
positive_words = words_from_file(fm.collection('positive-words.txt'))
negative_words = words_from_file(fm.collection('negative-words.txt'), encoding='ISO-8859-1')

# Subjclues

Used vim to transfer them to JSON; removed 'polarity' and 'mpqpolarity' since only one entry uses these.

In [6]:
import pandas as pd
subjclues = pd.read_json(fm.collection('subjclues.json'))

In [7]:
subjclues.head()

,len,pos1,priorpolarity,stemmed1,type,word1
0,1,adj,negative,n,weaksubj,abandoned
1,1,noun,negative,n,weaksubj,abandonment
2,1,verb,negative,y,weaksubj,abandon
3,1,verb,negative,y,strongsubj,abase
4,1,anypos,negative,y,strongsubj,abasement


In [54]:
import sklearn.feature_extraction as fe

sentence = "this is a test"
dv = fe.DictVectorizer()

In [57]:
import keras.preprocessing.text as kpt
tok = kpt.Tokenizer()
t1, t2 = "this is the first text", "some other second document"
tok.fit_on_texts([t1,t2])

In [65]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split

x, y, vocabulary, vocabulary_inv = load_data()
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42)

sequence_length = x.shape[1]
vocabulary_size = len(vocabulary_inv)
embedding_dim = 256
filter_sizes = [1, 1]
num_filters = 512
drop = 0.5

epochs = 100
batch_size = 30

# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(
    input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(
    num_filters, kernel_size=(filter_sizes[0], embedding_dim), 
    padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(
    num_filters, kernel_size=(filter_sizes[1], embedding_dim), 
    padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(
    num_filters, kernel_size=(filter_sizes[2], embedding_dim), 
    padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(
    sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(
    sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(
    sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint(
    'weights.{epoch:03d}-{val_acc:.4f}.hdf5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, 
    callbacks=[checkpoint], validation_data=(X_test, y_test))

['what', 'is', 'this']